In [2]:
import pandas as pd
import numpy as np
import sys, os
import sys
sys.path.insert(0,'/Applications/py3samsdk-master')
from py3samsdk.sscapi import PySSC

In [3]:
ssc_lib = '/Users/YONGJUN_SONG/Desktop/Cal-RAEL/Coding/SAM/sam-sdk-2015-6-30-r3/osx64/'  # path to SAM SSC Library
ssc = PySSC(ssc_lib)

In [4]:
test_data = pd.read_csv('/Users/YONGJUN_SONG/Desktop/test_0922.csv')

In [5]:
lat=["{:.4f}".format(x) for x in test_data['latitude']]
lng=["{:.4f}".format(y) for y in test_data['longitude']]
#years=['{:d}'.format(i) for i in xrange(2005, 2010)]
lat, lng
#for x in lat:  
#    for y in lng:  
#        t.append((x, y)) 
#test_data['lat_lng'] = test_data[['latitude', 'longitude']].apply(lambda x: ''.join(x), axis=1)

(['32.4772', '32.4743', '32.4754'], ['-86.4901', '-86.4734', '-86.4602'])

In [ ]:
db_test=[]
db_avg_test=[]
for x, y in zip(lat, lng):
    lat, lon = float(x), float(y)
    years=['{:d}'.format(i) for i in xrange(2005, 2007)]
    for i in years:
        year = i
        api_key = 'xgqyel6xB9OrU6cK7wORJIRqtpjU4QNgS9SjYjiP'
        attributes = 'ghi,dhi,dni,wind_speed_10m_nwp,surface_air_temperature_nwp,solar_zenith_angle'
        year = i
        leap_year = 'false'
        interval = '60'
        utc = 'false'
        your_email = 'yongjun.song@berkeley.edu'
        mailing_list = 'false'

        url = 'http://developer.nrel.gov/api/solar/nsrdb_0512_download.csv?wkt=POINT({lon}%20{lat})&names={year}&leap_day={leap}&interval={interval}&utc={utc}&email={email}&mailing_list={mailing_list}&api_key={api}&attributes={attr}'.format(year=year, lat=lat, lon=lon, leap=leap_year, interval=interval, utc=utc, email=your_email, mailing_list=mailing_list, api=api_key, attr=attributes)
        info = pd.read_csv(url, nrows=1)
        timezone, elevation = info['Local Time Zone'], info['Elevation']

        df = pd.read_csv('http://developer.nrel.gov/api/solar/nsrdb_0512_download.csv?wkt=POINT({lon}%20{lat})&names={year}&leap_day={leap}&interval={interval}&utc={utc}&email={email}&mailing_list={mailing_list}&api_key={api}&attributes={attr}'.format(year=year, lat=lat, lon=lon, leap=leap_year, interval=interval, utc=utc, email=your_email, mailing_list=mailing_list, api=api_key, attr=attributes), skiprows=2)

        df = df.set_index(pd.date_range('1/1/{yr}'.format(yr=year), freq=interval+'Min', periods=525600/int(interval)))

        wfd = ssc.data_create()
        ssc.data_set_number(wfd, 'lat', lat)
        ssc.data_set_number(wfd, 'lon', lon)
        ssc.data_set_number(wfd, 'tz', timezone)
        ssc.data_set_number(wfd, 'elev', elevation)
        ssc.data_set_array(wfd, 'year', df.index.year)
        ssc.data_set_array(wfd, 'month', df.index.month)
        ssc.data_set_array(wfd, 'day', df.index.day)
        ssc.data_set_array(wfd, 'hour', df.index.hour)
        ssc.data_set_array(wfd, 'minute', df.index.minute)
        ssc.data_set_array(wfd, 'dn', df['DNI'])
        ssc.data_set_array(wfd, 'df', df['DHI'])
        ssc.data_set_array(wfd, 'wspd', df['Wind Speed'])
        ssc.data_set_array(wfd, 'tdry', df['Temperature'])

        # Create SAM compliant object  
        dat = ssc.data_create()
        ssc.data_set_table(dat, 'solar_resource_data', wfd)
        ssc.data_free(wfd)

        # Specify the system Configuration
        # Set system capacity in MW
        system_capacity = 4
        ssc.data_set_number(dat, 'system_capacity', system_capacity)
        # Set DC/AC ratio (or power ratio). See https://sam.nrel.gov/sites/default/files/content/virtual_conf_july_2013/07-sam-virtual-conference-2013-woodcock.pdf
        ssc.data_set_number(dat, 'dc_ac_ratio', 1.15)
        # Set tilt of system in degrees
        # For Google data, roof segments are considered Flat for roofs with a tilt of less than 10%
        ssc.data_set_number(dat, 'tilt',0)
        # Set azimuth angle (in degrees) from north (0 degrees)
        ssc.data_set_number(dat, 'azimuth', 180)
        # Set the inverter efficency
        ssc.data_set_number(dat, 'inv_eff', 96)
        # Set the system losses, in percent
        ssc.data_set_number(dat, 'losses', 14.0757)
        # Specify fixed tilt system (0=Fixed, 1=Fixed Roof, 2=1 Axis Tracker, 3=Backtracted, 4=2 Axis Tracker)
        ssc.data_set_number(dat, 'array_type', 0)
        # Set ground coverage ratio
        ssc.data_set_number(dat, 'gcr', 1)
        # Set constant loss adjustment
        ssc.data_set_number(dat, 'adjust:constant', 0)

        # execute and put generation results back into dataframe
        mod = ssc.module_create('pvwattsv5')
        ssc.module_exec(mod, dat)
        df['generation'] = np.array(ssc.data_get_array(dat, 'gen'))

        # free the memory
        ssc.data_free(dat)
        ssc.module_free(mod)

        db_test.append(df['generation'].sum())
    avg=sum(db_test)/len(db_test)
    db_avg_test.append(avg)
   
    ssc.data_free(dat)
    ssc.module_free(mod)
        #db_multiple_location.append(df['generation_avg_ten']())

In [ ]:
db_avg_test

In [33]:
db_multiple_location=[]
for x, y in zip(lat, lng):
    
    lat, lon, year = float(x), float(y), 2006
    api_key = 'xgqyel6xB9OrU6cK7wORJIRqtpjU4QNgS9SjYjiP'
    attributes = 'ghi,dhi,dni,wind_speed_10m_nwp,surface_air_temperature_nwp,solar_zenith_angle'
    year = 2006
    leap_year = 'false'
    interval = '60'
    utc = 'false'
    your_email = 'yongjun.song@berkeley.edu'
    mailing_list = 'false'

    url = 'http://developer.nrel.gov/api/solar/nsrdb_0512_download.csv?wkt=POINT({lon}%20{lat})&names={year}&leap_day={leap}&interval={interval}&utc={utc}&email={email}&mailing_list={mailing_list}&api_key={api}&attributes={attr}'.format(year=year, lat=lat, lon=lon, leap=leap_year, interval=interval, utc=utc, email=your_email, mailing_list=mailing_list, api=api_key, attr=attributes)
    info = pd.read_csv(url, nrows=1)
    timezone, elevation = info['Local Time Zone'], info['Elevation']

    df = pd.read_csv('http://developer.nrel.gov/api/solar/nsrdb_0512_download.csv?wkt=POINT({lon}%20{lat})&names={year}&leap_day={leap}&interval={interval}&utc={utc}&email={email}&mailing_list={mailing_list}&api_key={api}&attributes={attr}'.format(year=year, lat=lat, lon=lon, leap=leap_year, interval=interval, utc=utc, email=your_email, mailing_list=mailing_list, api=api_key, attr=attributes), skiprows=2)

    df = df.set_index(pd.date_range('1/1/{yr}'.format(yr=year), freq=interval+'Min', periods=525600/int(interval)))

    wfd = ssc.data_create()
    ssc.data_set_number(wfd, 'lat', lat)
    ssc.data_set_number(wfd, 'lon', lon)
    ssc.data_set_number(wfd, 'tz', timezone)
    ssc.data_set_number(wfd, 'elev', elevation)
    ssc.data_set_array(wfd, 'year', df.index.year)
    ssc.data_set_array(wfd, 'month', df.index.month)
    ssc.data_set_array(wfd, 'day', df.index.day)
    ssc.data_set_array(wfd, 'hour', df.index.hour)
    ssc.data_set_array(wfd, 'minute', df.index.minute)
    ssc.data_set_array(wfd, 'dn', df['DNI'])
    ssc.data_set_array(wfd, 'df', df['DHI'])
    ssc.data_set_array(wfd, 'wspd', df['Wind Speed'])
    ssc.data_set_array(wfd, 'tdry', df['Temperature'])

    # Create SAM compliant object  
    dat = ssc.data_create()
    ssc.data_set_table(dat, 'solar_resource_data', wfd)
    ssc.data_free(wfd)

    # Specify the system Configuration
    # Set system capacity in MW
    system_capacity = 4
    ssc.data_set_number(dat, 'system_capacity', system_capacity)
    # Set DC/AC ratio (or power ratio). See https://sam.nrel.gov/sites/default/files/content/virtual_conf_july_2013/07-sam-virtual-conference-2013-woodcock.pdf
    ssc.data_set_number(dat, 'dc_ac_ratio', 1.15)
    # Set tilt of system in degrees
    # For Google data, roof segments are considered Flat for roofs with a tilt of less than 10%
    ssc.data_set_number(dat, 'tilt',0)
    # Set azimuth angle (in degrees) from north (0 degrees)
    ssc.data_set_number(dat, 'azimuth', 180)
    # Set the inverter efficency
    ssc.data_set_number(dat, 'inv_eff', 96)
    # Set the system losses, in percent
    ssc.data_set_number(dat, 'losses', 14.0757)
    # Specify fixed tilt system (0=Fixed, 1=Fixed Roof, 2=1 Axis Tracker, 3=Backtracted, 4=2 Axis Tracker)
    ssc.data_set_number(dat, 'array_type', 0)
    # Set ground coverage ratio
    ssc.data_set_number(dat, 'gcr', 1)
    # Set constant loss adjustment
    ssc.data_set_number(dat, 'adjust:constant', 0)

    # execute and put generation results back into dataframe
    mod = ssc.module_create('pvwattsv5')
    ssc.module_exec(mod, dat)
    df['generation2006'] = np.array(ssc.data_get_array(dat, 'gen'))

    # free the memory
    ssc.data_free(dat)
    ssc.module_free(mod)

    db_multiple_location.append(df['generation2006'].sum())

In [203]:
years=['{:d}'.format(x) for x in xrange(2005, 2010)]
db_PDM=[]
for x in years:
    lat, lon, year = 37.8216, -122.2253, x
    api_key = 'xgqyel6xB9OrU6cK7wORJIRqtpjU4QNgS9SjYjiP'
    attributes = 'ghi,dhi,dni,wind_speed_10m_nwp,surface_air_temperature_nwp,solar_zenith_angle'
    year = x
    leap_year = 'false'
    interval = '60'
    utc = 'false'
    your_email = 'yongjun.song@berkeley.edu'
    mailing_list = 'false'

    url = 'http://developer.nrel.gov/api/solar/nsrdb_0512_download.csv?wkt=POINT({lon}%20{lat})&names={year}&leap_day={leap}&interval={interval}&utc={utc}&email={email}&mailing_list={mailing_list}&api_key={api}&attributes={attr}'.format(year=year, lat=lat, lon=lon, leap=leap_year, interval=interval, utc=utc, email=your_email, mailing_list=mailing_list, api=api_key, attr=attributes)
    info = pd.read_csv(url, nrows=1)
    timezone, elevation = info['Local Time Zone'], info['Elevation']

    df = pd.read_csv('http://developer.nrel.gov/api/solar/nsrdb_0512_download.csv?wkt=POINT({lon}%20{lat})&names={year}&leap_day={leap}&interval={interval}&utc={utc}&email={email}&mailing_list={mailing_list}&api_key={api}&attributes={attr}'.format(year=year, lat=lat, lon=lon, leap=leap_year, interval=interval, utc=utc, email=your_email, mailing_list=mailing_list, api=api_key, attr=attributes), skiprows=2)

    df = df.set_index(pd.date_range('1/1/{yr}'.format(yr=year), freq=interval+'Min', periods=525600/int(interval)))



    wfd = ssc.data_create()
    ssc.data_set_number(wfd, 'lat', lat)
    ssc.data_set_number(wfd, 'lon', lon)
    ssc.data_set_number(wfd, 'tz', timezone)
    ssc.data_set_number(wfd, 'elev', elevation)
    ssc.data_set_array(wfd, 'year', df.index.year)
    ssc.data_set_array(wfd, 'month', df.index.month)
    ssc.data_set_array(wfd, 'day', df.index.day)
    ssc.data_set_array(wfd, 'hour', df.index.hour)
    ssc.data_set_array(wfd, 'minute', df.index.minute)
    ssc.data_set_array(wfd, 'dn', df['DNI'])
    ssc.data_set_array(wfd, 'df', df['DHI'])
    ssc.data_set_array(wfd, 'wspd', df['Wind Speed'])
    ssc.data_set_array(wfd, 'tdry', df['Temperature'])

    # Create SAM compliant object  
    dat = ssc.data_create()
    ssc.data_set_table(dat, 'solar_resource_data', wfd)
    ssc.data_free(wfd)

    # Specify the system Configuration
    # Set system capacity in MW
    system_capacity = 4
    ssc.data_set_number(dat, 'system_capacity', system_capacity)
    # Set DC/AC ratio (or power ratio). See https://sam.nrel.gov/sites/default/files/content/virtual_conf_july_2013/07-sam-virtual-conference-2013-woodcock.pdf
    ssc.data_set_number(dat, 'dc_ac_ratio', 1.15)
    # Set tilt of system in degrees
    # For Google data, roof segments are considered Flat for roofs with a tilt of less than 10%
    ssc.data_set_number(dat, 'tilt',0)
    # Set azimuth angle (in degrees) from north (0 degrees)
    ssc.data_set_number(dat, 'azimuth', 180)
    # Set the inverter efficency
    ssc.data_set_number(dat, 'inv_eff', 96)
    # Set the system losses, in percent
    ssc.data_set_number(dat, 'losses', 14.0757)
    # Specify fixed tilt system (0=Fixed, 1=Fixed Roof, 2=1 Axis Tracker, 3=Backtracted, 4=2 Axis Tracker)
    ssc.data_set_number(dat, 'array_type', 0)
    # Set ground coverage ratio
    ssc.data_set_number(dat, 'gcr', 1)
    # Set constant loss adjustment
    ssc.data_set_number(dat, 'adjust:constant', 0)

    # execute and put generation results back into dataframe
    mod = ssc.module_create('pvwattsv5')
    ssc.module_exec(mod, dat)
    df['generation'+'x'] = np.array(ssc.data_get_array(dat, 'gen'))

    # free the memory
    ssc.data_free(dat)
    ssc.module_free(mod)

    db_PDM.append(df['generation'+'x'].sum())
    
db_PDM

[5367.807927011971,
 5281.871743804659,
 5582.04845462463,
 5593.582307275938,
 5479.782530828961]